# Implement R2D2 algorithm in small scale

In [1]:
import os, sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import random
import numpy as np

import math

%load_ext autoreload
%autoreload 2

In [11]:
from methods.r2d2_utils.lrd2_labels import make_float_label as label_lrd2
from methods.r2d2_utils.r2d2_labels import make_float_label as label_r2d2, make_long_label as label_r2d2_long

In [12]:
label_r2d2_long(5,4)

tensor([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
       device='cuda:0')

In [9]:
label_r2d2(4,5)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.]], device='cuda:0')

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
os.getcwd()

'/home/asus/DL_Fewshot_Project/fewshotbench'

In [6]:
dlr_path = os.path.join(os.getcwd(), 'data', 'test', 'train_loader.pt')
# read in the data loader
train_loader = torch.load(dlr_path)

  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2023-06-11) 46,420 Terms; optional_attrs(relationship)


## A train loop

In [5]:
n_way = 5
n_support = 5
n_query = 15
change_way = True

In [7]:
# length of the data loader
len(train_loader)

100

In [9]:
# a train loop
for i, (x, _) in enumerate(train_loader):
    if isinstance(x, list):
        n_query = x[0].size(1) - n_support
        if change_way:
            n_way = x[0].size(0)
    else: 
        n_query = x.size(1) - n_support
        if change_way:
            n_way = x.size(0)
    
    print('i: ', i)
    print('n_way: ', n_way)
    print('n_support: ', n_support)
    print('n_query: ', n_query)
    print('x: ', x.shape) # shape (n_way, n_support + n_query, **embed)


/home/asus/miniconda3/envs/fewshotbench/lib/python3.10/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


i:  0
n_way:  5
n_support:  5
n_query:  15
x:  torch.Size([5, 20, 1280])
i:  1
n_way:  5
n_support:  5
n_query:  15
x:  torch.Size([5, 20, 1280])
i:  2
n_way:  5
n_support:  5
n_query:  15
x:  torch.Size([5, 20, 1280])
i:  3
n_way:  5
n_support:  5
n_query:  15
x:  torch.Size([5, 20, 1280])
i:  4
n_way:  5
n_support:  5
n_query:  15
x:  torch.Size([5, 20, 1280])
i:  5
n_way:  5
n_support:  5
n_query:  15
x:  torch.Size([5, 20, 1280])
i:  6
n_way:  5
n_support:  5
n_query:  15
x:  torch.Size([5, 20, 1280])
i:  7
n_way:  5
n_support:  5
n_query:  15
x:  torch.Size([5, 20, 1280])
i:  8
n_way:  5
n_support:  5
n_query:  15
x:  torch.Size([5, 20, 1280])
i:  9
n_way:  5
n_support:  5
n_query:  15
x:  torch.Size([5, 20, 1280])
i:  10
n_way:  5
n_support:  5
n_query:  15
x:  torch.Size([5, 20, 1280])
i:  11
n_way:  5
n_support:  5
n_query:  15
x:  torch.Size([5, 20, 1280])
i:  12
n_way:  5
n_support:  5
n_query:  15
x:  torch.Size([5, 20, 1280])
i:  13
n_way:  5
n_support:  5
n_query:  15
x:  

In [ ]:
is_feature = True
backbone = 
feature = backbone

In [ ]:
if isinstance(x, list):
    x = [Variable(obj.to(device)) for obj in x]
else: x = Variable(x.to(device))
if is_feature:
    z_all = x
else:
    if isinstance(x, list):
        x = [obj.contiguous().view(n_way * (n_support + n_query), *obj.size()[2:]) for obj in x]
    else: x = x.contiguous().view(n_way * (n_support + n_query), *x.size()[2:])
    z_all = feature.forward(x)
    z_all = z_all.view(n_way, n_support + n_query, -1)
z_support = z_all[:, :n_support]
z_query = z_all[:, n_support:]

## Other testing

In [15]:
labels.make_float_label(5,4)

tensor([[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.]], device='cuda:0')